# Selenium을 이용한 아산병원 질환 링크 크롤링 (1페이지)
병명 상세페이지 링크를 수집합니다.

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

from bs4 import BeautifulSoup
import pandas as pd

In [2]:
# 셀레니움 설정
options = Options()
options.add_argument("--headless")  # 창 없이 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# chromedriver가 PATH에 있다면 아래처럼 실행
service = Service()
driver = webdriver.Chrome(service=service, options=options)

# 페이지 접속
driver.get(
    "https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseList.do?pageIndex=1"
)
time.sleep(3)  # 렌더링 대기

In [3]:
# 질환 상세 링크 수집
elements = driver.find_elements(
    By.CSS_SELECTOR, "#listForm ul li div.contBox > strong > a"
)
disease_links = [el.get_attribute("href") for el in elements]

print(f"총 {len(disease_links)}개의 링크를 찾음")

총 20개의 링크를 찾음


## 상세페이지 20개에서 병명, 증상, 진료과 크롤링
- 병명 selector: `#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > strong`
- 증상 selector: `#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.contDescription > dl > dd:nth-child(6) > p`
- 진료과 selector: `#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > dl > dd:nth-child(6) > a`

In [8]:
from bs4 import BeautifulSoup
import pandas as pd
import time

results = []

for link in [disease_links[3]]:
    try:
        driver.get(link)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 병명
        title_tag = soup.select_one('#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > strong')
        title = title_tag.get_text(strip=True) if title_tag else ''

        # 증상
        symptom_tag = soup.select_one('#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.contDescription > dl > dd:nth-child(6) > p')
        symptom = symptom_tag.get_text(strip=True) if symptom_tag else ''

        # 진료과
        dept_tag = soup.select_one('#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > dl > dd:nth-child(6) > a')
        dept = dept_tag.get_text(strip=True) if dept_tag else ''

        results.append({'병명': title, '증상': symptom, '진료과': dept})
        print(f"[완료] {title}")
    except Exception as e:
        print(f"[에러] {link} - {e}")
        continue

[완료] 4번 염색체 단완결실 증후군(Wolf-Hirschhorn syndrome)


In [9]:
results

[{'병명': '4번 염색체 단완결실 증후군(Wolf-Hirschhorn syndrome)',
  '증상': '4번 염색체 단완결실 증후군은 그리스 투구형 얼굴(장두증, 전두골 돌출, 뚜렷하고 넓은 미간, 안구의 저형성, 내안각췌피, 크고 넓은 코, 뾰족한 코, 소하악증)이 특징입니다. 또한 사시, 낮게 변형된 귀, 아래로 처진 입, 구순열 및 구개열(15%), 심장 기형(중격 결손), 요도 하열, 잠복 고환, 척추 측만증, 발작 등의 증상이 동반됩니다. 경중의 차이가 있기는 하지만 정신 지체, 수유 곤란의 증세가 나타나기도 합니다.',
  '진료과': '의학유전학센터'}]

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.amc.seoul.kr/asan/healthinfo/disease/diseaseDetail.do?contentId=32329"
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# 셀렉터로 내용 추출
title_tag = soup.select_one("#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > strong")
symptom_tag = soup.select_one("#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.contDescription > dl > dd:nth-child(6) > p")
dept_tag = soup.select_one("#content > div.healthinfoWrap.clearfix > div.regionReviewLeft > div.otherRegionBox > ul > li > div.contBox > dl > dd:nth-child(6) > a")

# 출력
print("질환명:", title_tag.get_text(strip=True) if title_tag else "❌ 없음")
print("증상 내용:", symptom_tag.get_text(strip=True) if symptom_tag else "❌ 없음")
print("진료과:", dept_tag.get_text(strip=True) if dept_tag else "❌ 없음")


In [ ]:
df = pd.DataFrame(results)
df.to_csv("amc_disease_20pages.csv", index=False, encoding="utf-8-sig")
print("✅ CSV 저장 완료")